In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [20]:
def normalize_list(values):
    # Filter out None values for min-max calculation
    clean_values = [v for v in values if v is not None]
    min_val = min(clean_values)
    max_val = max(clean_values)
    
    # Avoid division by zero if all values are the same
    def normalize(v):
        if v is None:
            return None
        elif max_val == min_val:
            return 0.5
        else:
            return (v - min_val) / (max_val - min_val)
    
    return [normalize(v) for v in values]

In [30]:
# print(normalize_list([92, 390, 600, 780]))

In [29]:
data = {
    'Metric': ['Avg. activated rules/arg', 'Avg. cond. per rule/arg', 'Total att.', 'Avg. activated att.'] * 13,
    # 'Value': [1, 1, None, None, 1, 1, 0.0, 0.14, 1, 1, 0.43, 0.46, 1, 1, 0.73, 0.70, 1, 1, 1, 0.93, 1, 1, 0.0, 0.14, 1, 1, 0.43, 0.46, 1, 1, 0.73, 0.70, 1, 1, 1, 0.93, 1, 1, 0.0, 0.14, 1, 1, 0.43, 0.46, 1, 1, 0.73, 0.70, 1, 1, 1, 0.93], 
    'Value': [1.88, 2.45, None, None, 1.88, 2.45, 92, 0.14, 1.88, 2.45, 390, 0.46, 1.88, 2.45, 600, 0.70, 
              1.88, 2.45, 780, 0.93, 1.88, 2.45, 92, 0.14, 1.88, 2.45, 390, 0.46, 1.88, 2.45, 600, 0.70, 
              1.88, 2.45, 780, 0.93, 1.88, 2.45, 92, 0.14, 1.88, 2.45, 390, 0.46, 1.88, 2.45, 600, 0.70, 
              1.88, 2.45, 780, 0.93],
    'Model': ['LLM1'] * 4 + ['G25'] * 4 + ['G50'] * 4 + ['G90'] * 4 + ['G100'] * 4 + 
             ['P25'] * 4 + ['P50'] * 4 + ['P90'] * 4 + ['P100'] * 4 + 
             ['C25'] * 4 + ['C50'] * 4 + ['C90'] * 4 + ['C100'] * 4
}
df = pd.DataFrame(data)

In [26]:
df = pd.DataFrame(data)

# Create radar chart
fig = px.line_polar(df, r='Value', theta='Metric', color='Model', line_close=True)
fig.update_traces(fill='toself')
fig.show()

In [33]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Pivot the data
pivot = df.pivot(index='Model', columns='Metric', values='Value')

# Pick your 4 metrics
metrics = pivot.columns.tolist()

# Create 2x2 subplots
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=metrics,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.15)

# Add each metric as a separate heatmap
for i, metric in enumerate(metrics):
    row = i // 2 + 1
    col = i % 2 + 1
    z = pivot[[metric]]
    fig.add_trace(
        go.Heatmap(
            z=z.values,
            x=[metric],
            y=z.index,
            colorscale='Viridis',
            colorbar=dict(title=metric, len=0.3, y=1 - (i * 0.25)),  # Position colorbars
        ),
        row=row, col=col
    )

fig.update_layout(
    height=600,
    width=800,
    title_text="Separate Heatmaps by Metric",
    showlegend=False
)

fig.show()


In [32]:
# Pivot the original data
heatmap_data = df.pivot(index='Model', columns='Metric', values='Value')

# Normalize each column (metric) individually
normalized = heatmap_data.copy()
for col in normalized.columns:
    col_min = normalized[col].min()
    col_max = normalized[col].max()
    if col_max != col_min:
        normalized[col] = (normalized[col] - col_min) / (col_max - col_min)
    else:
        normalized[col] = 0.5  # Flat values → neutral middle color

# Plot normalized heatmap
fig = px.imshow(normalized, color_continuous_scale='Viridis', aspect='auto')
fig.update_layout(title='Metric-wise Normalized Heatmap')
fig.show()